In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Layer, MaxPooling2D, Flatten, Dense, Conv2D, Input, BatchNormalization, MaxPool2D, Lambda, Concatenate, Dropout
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import initializers
import tensorflow.keras.backend as K
import numpy as np
%load_ext tensorboard
import datetime
!rm -rf ./logs/ 

### Make The Dataset

In [ ]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255
x_test = x_test / 255

# y_train = keras.utils.to_categorical(y_train, 10)
# y_test = keras.utils.to_categorical(y_test, 10)

image_size = (32,32,3)

# flip images
x_train_flipped = np.flip(x_train, axis=3)
x_train = np.append(x_train, x_train_flipped, axis=0)
y_train = np.append(y_train, np.copy(y_train), axis=0)
x_train_flipped = None
print('flipped')

# put in CEDA
# rands = (np.random.rand(len(x_train)//10,32,32,3)*255).astype(int)
# yrands = np.tile(-1, len(x_train)//10).reshape(len(x_train)//10, 1)
# y_train = np.append(y_train, yrands, axis=0)
# x_train = np.append(x_train, rands, axis=0)
# rands = None
# yrands = None
# print('CEDA done')

# mix it up
x_train, y_train = unison_shuffled_copies(x_train, y_train)
print('permuted')
print(x_train.shape, y_train.shape)

170500096/170498071 [==============================] - 11s 0us/step
flipped
permuted
(100000, 32, 32, 3) (100000, 1)


In [ ]:
def make_set(nums):
  ytrain = np.zeros((y_train.shape[0], len(nums)))
  ytest = np.zeros((y_test.shape[0], len(nums)))

  for i in range(y_train.shape[0]):
    arg = y_train[i][0]
    for x in range(len(nums)):
      if arg in nums[x]:
        ytrain[i][x] = 1.0
        made1 = True

  for i in range(y_test.shape[0]):
    arg = y_train[i][0]
    for x in range(len(nums)):
      if arg in nums[x]:
        ytest[i][x] = 1.0
        made1 = True
  
  return ytrain, ytest

In [ ]:
l1 = 1e-4
l2 = 5e-4
def make_class(cin, filters, classes):
  c = Conv2D(filters, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(cin)
  c = MaxPool2D()(c)
  c = Conv2D(filters, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(c)
  c = Conv2D(filters*2, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(c)
  c = MaxPool2D()(c)
  c = Flatten()(c)
  c = Dense(128, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(c)
  c = Dense(classes, activation="sigmoid")(c)
  return c

def MPU(in_shape, out_classes, name=None, class_only = False):
  inp = Input(shape=in_shape)
  shared = Conv2D(64, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(inp)

  classifier = make_class(shared, 64, out_classes)

  if class_only: return Model(inp, classifier, name=name)

  extra = Conv2D(64, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(shared)
  extra = BatchNormalization()(extra)
  extra = Dropout(0.5)(extra)

  return Model(inp, [classifier, extra], name=name)

# seq = Sequential([
#   Input(shape=(32,32,3)),
#   Conv2D(64, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)),
#   Conv2D(64, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)),
#   BatchNormalization(),
#   Dropout(0.5),
#   Conv2D(128, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)),
#   Conv2D(128, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)),
#   BatchNormalization(),
#   Dropout(0.5),
#   Conv2D(192, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)),
#   Conv2D(192, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)),
#   BatchNormalization(),
#   Dropout(0.5),
#   Conv2D(192, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)),
#   Conv2D(192, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)),
#   MaxPool2D(),
#   Conv2D(192, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)),
#   Conv2D(384, (3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)),
#   MaxPool2D(),
#   Flatten(),
#   Dense(256, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)),
#   Dense(10, activation="softmax")
# ])

In [ ]:
seq.summary()

In [ ]:
sgd = optimizers.SGD(learning_rate=0.0005, momentum=0.9, nesterov=True)
seq.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
seq.fit(x_train, y_train, batch_size=64, epochs=30, validation_data=(x_test, y_test))

In [ ]:
import functools
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'
seq.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy',top3_acc])
# ])

In [ ]:
seq.evaluate(x_test, y_test)

313/313 [==============================] - 3s 10ms/step - loss: 2.3066 - accuracy: 0.7279 - top3_acc: 0.9291


[2.306570291519165, 0.7279000282287598, 0.929099977016449]

In [ ]:
root = MPU((32,32,3), 2, name="root")
root.summary()
root.save('rootActual.hdf5')
nalive = MPU(root.outputs[1].shape[1:], 3, name="MPU A")

# nalive.summary()
alive = MPU(root.outputs[1].shape[1:], 4, name="alive")
road = MPU(nalive.outputs[1].shape[1:], 2, name="road", class_only=True)
pet = MPU(alive.outputs[1].shape[1:], 2, name="pet", class_only=True)
medium = MPU(alive.outputs[1].shape[1:], 2, name="medium", class_only=True)

Model: "root"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_108 (Conv2D)             (None, 32, 32, 64)   1792        input_25[0][0]                   
__________________________________________________________________________________________________
conv2d_109 (Conv2D)             (None, 32, 32, 64)   36928       conv2d_108[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_48 (MaxPooling2D) (None, 16, 16, 64)   0           conv2d_109[0][0]                 
_______________________________________________________________________________________________

all the models

In [ ]:
imp = Input(shape=(32,32,3))
rootM = root(imp)
naliveM = nalive(rootM[1])
aliveM = alive(rootM[1])
roadM = road(naliveM[1])
petM = pet(aliveM[1])
mediumM = medium(aliveM[1])

full = Model(imp, [rootM[0], naliveM[0], aliveM[0], roadM, petM, mediumM], name="full")

rooti = Model(imp, rootM[0], name="root_individual")
nalivei = Model(imp, naliveM[0], name="nalive_individual")
alivei = Model(imp, aliveM[0], name="alive_individual")
roadi = Model(imp, roadM, name="road_individual")
peti = Model(imp, petM, name="pet_individual")
mediumi = Model(imp, mediumM, name="medium_individual")

In [ ]:
full.summary()

Model: "full"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
root (Model)                    [(None, 2), (None, 3 1198722     input_14[0][0]                   
__________________________________________________________________________________________________
nalive (Model)                  [(None, 3), (None, 3 1233987     root[1][1]                       
__________________________________________________________________________________________________
alive (Model)                   [(None, 4), (None, 3 1234116     root[1][1]                       
_______________________________________________________________________________________________

dataset ys for the models

In [ ]:
rootD_train, rootD_test = make_set([[0, 1, 8, 9], [2, 3, 4, 5, 6]])
naliveD_train, naliveD_test = make_set([[0], [8], [1, 9]])
aliveD_train, aliveD_test = make_set([[2], [6], [3, 5], [4, 7]])
roadD_train, roadD_test = make_set([[1], [9]])
petD_train, petD_test = make_set([[3], [5]])
mediumD_train, mediumD_test = make_set([[4], [7]])

In [ ]:
sgd = optimizers.SGD(learning_rate=0.0005, momentum=0.9, nesterov=True)
full.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
rooti.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
nalivei.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
alivei.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
roadi.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
peti.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
mediumi.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
!rm -rf ./full/ 

In [ ]:
full.fit(x_train, [rootD_train, naliveD_train, aliveD_train, roadD_train, petD_train, mediumD_train], epochs=30, batch_size=64,validation_data=(x_test, [rootD_test, naliveD_test, aliveD_test, roadD_test, petD_test, mediumD_test]))

Epoch 1/30
1563/1563 [==============================] - 78s 50ms/step - loss: 15.1166 - root_loss: 0.5337 - nalive_loss: 0.3100 - alive_loss: 0.6866 - road_loss: 0.1292 - pet_loss: 0.1333 - medium_loss: 0.1178 - root_accuracy: 0.6482 - nalive_accuracy: 0.4004 - alive_accuracy: 0.3113 - road_accuracy: 0.6447 - pet_accuracy: 0.5786 - medium_accuracy: 0.4903 - val_loss: 15.5651 - val_root_loss: 0.7790 - val_nalive_loss: 0.5605 - val_alive_loss: 0.9526 - val_road_loss: 0.1464 - val_pet_loss: 0.1431 - val_medium_loss: 0.1574 - val_root_accuracy: 0.5088 - val_nalive_accuracy: 0.2216 - val_alive_accuracy: 0.2189 - val_road_accuracy: 0.5387 - val_pet_accuracy: 0.6909 - val_medium_accuracy: 0.7166
Epoch 2/30
1563/1563 [==============================] - 77s 49ms/step - loss: 14.0040 - root_loss: 0.3748 - nalive_loss: 0.2442 - alive_loss: 0.5775 - road_loss: 0.1235 - pet_loss: 0.1286 - medium_loss: 0.0988 - root_accuracy: 0.7597 - nalive_accuracy: 0.4891 - alive_accuracy: 0.3998 - road_accuracy: 

In [ ]:
# full.load_weights('fullweights.hdf5')
full.evaluate(x_test, [rootD_test, naliveD_test, aliveD_test, roadD_test, petD_test, mediumD_test])

313/313 [==============================] - 4s 12ms/step - loss: 8.9735 - root_loss: 1.7009 - nalive_loss: 1.4836 - alive_loss: 3.2882 - road_loss: 0.2541 - pet_loss: 0.2075 - medium_loss: 0.2836 - root_accuracy: 0.5028 - nalive_accuracy: 0.3403 - alive_accuracy: 0.2331 - road_accuracy: 0.4846 - pet_accuracy: 0.6802 - medium_accuracy: 0.5645


[8.973502159118652,
 1.700938105583191,
 1.483573079109192,
 3.288203239440918,
 0.2540770471096039,
 0.20745068788528442,
 0.283590167760849,
 0.5027999877929688,
 0.3402999937534332,
 0.23309999704360962,
 0.4846000075340271,
 0.6801999807357788,
 0.5644999742507935]

In [ ]:
full.summary()

Model: "full"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
root (Model)                    [(None, 2), (None, 3 1198722     input_8[0][0]                    
__________________________________________________________________________________________________
nalive (Model)                  [(None, 3), (None, 3 1233987     root[1][1]                       
__________________________________________________________________________________________________
alive (Model)                   [(None, 4), (None, 3 1234116     root[1][1]                       
_______________________________________________________________________________________________

In [ ]:
%tensorboard --logdir ind

In [ ]:
ind_callback = tf.keras.callbacks.TensorBoard(log_dir="./ind")
full.trainable = False
rooti.trainable = True
alivei.trainable = False
print('rooti')
rooti.fit(x_train, rootD_train, epochs=30, batch_size=16, validation_data=(x_test, rootD_test), callbacks=[ind_callback])
nalivei.trainable = True
roadi.trainable = False
print('nalivei')
nalivei.fit(x_train, naliveD_train, epochs=30, batch_size=16, validation_data=(x_test, naliveD_test), callbacks=[ind_callback])
alivei.trainable = True
peti.trainable = False
print('alivei')
alivei.fit(x_train, aliveD_train, epochs=30, batch_size=16, validation_data=(x_test, aliveD_test), callbacks=[ind_callback])
roadi.trainable = True
nalivei.trainable = False
print('roadi')
roadi.fit(x_train, roadD_train, epochs=30, batch_size=16, validation_data=(x_test, roadD_test), callbacks=[ind_callback])
peti.trainable = True
alivei.trainable = False
print('peti')
peti.fit(x_train, petD_train, epochs=30, batch_size=16, validation_data=(x_test, petD_test), callbacks=[ind_callback])
mediumi.trainable = True
alivei.trainable = False
print('mediumi')
mediumi.fit(x_train, mediumD_train, epochs=30, batch_size=16, validation_data=(x_test, mediumD_test), callbacks=[ind_callback])

### Test The Networks

In [ ]:
tests = full.predict(x_test)

In [ ]:
abovep1 = 0
for i in tests[2]:
  if np.argmax(i) > 0:
    abovep1 += 1
print(abovep1, "/", len(tests[0]), '(', str(100*abovep1/len(tests[0])) + '%', ')')

8363 / 10000 ( 83.63% )


In [ ]:
import copy
hierarchy = {
    0: [[0, 0], [1, 0]],
    8: [[0, 0], [1, 1]],
    1: [[0, 0], [1, 2], [3, 0]],
    9: [[0, 0], [1, 2], [3, 1]],
    2: [[0, 1], [2, 0]],
    6: [[0, 1], [2, 1]],
    3: [[0, 1], [2, 2], [4, 0]],
    5: [[0, 1], [2, 2], [4, 1]],
    4: [[0, 1], [2, 3], [5, 0]],
    7: [[0, 1], [2, 3], [5, 1]]
}

total = 0
lt = len(tests[0])
threshold=0.3
for i in range(len(tests[0])):
  arg = y_test[i][0]
  pos = hierarchy[arg]
  passed = True

  finals = [tests[1][i][0], tests[1][i][1], tests[2][i][0], tests[2][i][1], tests[3][i][0], tests[3][i][1], tests[4][i][0], tests[4][i][1], tests[5][i][0], tests[5][i][1]]
  finals = copy.deepcopy([finals[0], finals[2], finals[4], finals[6], finals[8], finals[7], finals[5], finals[9], finals[1], finals[3]])

  # conditional

  if tests[0][i][0] < threshold:
    finals[0], finals[1], finals[8], finals[9] = 0, 0, 0, 0
  if tests[0][i][1] < threshold:
    finals[2], finals[3], finals[4], finals[5], finals[6], finals[7] = 0, 0, 0, 0, 0, 0
  if tests[1][i][0] < threshold: finals[0] = 0
  if tests[1][i][1] < threshold: finals[8] = 0
  if tests[1][i][2] < threshold: finals[1], finals[9] = 0, 0
  if tests[2][i][0] < threshold: finals[2] = 0
  if tests[2][i][1] < threshold: finals[6] = 0
  if tests[2][i][2] < threshold: finals[3], finals[5] = 0, 0
  if tests[2][i][3] < threshold: finals[4], finals[7] = 0, 0

  if np.argmax(finals) == arg and any(finals):
    total += 1

  # full

  # print(arg, finals)
  # if np.argmax(finals) == arg: total += 1
  # indices = (-np.array(finals)).argsort()[:3]
  # if arg in indices: total += 1

  # single

  
  # for p in pos:
  #   if np.argmax(tests[p[0]][i]) != p[1]: 
  #     passed = False
  #     break
  # if passed: total += 1
  # break

print(total/lt)




0.0643


single-route accuracy

In [ ]:
count = 0
total = len(x_test)
for img in range(len(x_test)):
  levels = rootNode.run(x_test[img], connection_type=2)
  compare = levels[-1]
  # print(compare, y_test[img])
  if compare[0][1] == y_test[img][0]:
    count += 1
  if img%20 == 0:
    print('\r', img/total, count/(img+1), '    ', end='')

print(count/total)

conditional-execution accuracy with a threshold of 0.2

In [ ]:
count = 0
total = len(x_test)
for img in range(len(x_test)):
  levels = rootNode.run(x_test[img], connection_type=1, threshold=0.2)
  fines = []
  for i in levels:
    # print(i)
    if 'FINE' in i: fines.append(i)
    elif type(i[0]) == list:
      for x in i:
        if type(x) == list and 'FINE' in x:
          fines.append(x)

  # print(fines)
  fines.sort(reverse=True, key=lambda x: x[2])
  if len(fines) > 0 and fines[0][1] == y_test[img][0]:
    count += 1

  if img%20 == 0:
    print('\r', img/total, count/(img+1), '    ', end='')

print(count/total)

full-network accuracy

In [ ]:
count = 0
topkcount = 0
topk = 3
total = len(x_test)
for img in range(len(x_test)):
  levels = rootNode.run(x_test[img], connection_type=0)
  fines = []
  for i in levels:
    # print(i)
    if 'FINE' in i: fines.append(i)
    elif type(i[0]) == list:
      for x in i:
        if type(x) == list and 'FINE' in x:
          fines.append(x)

  # print(fines)
  fines.sort(reverse=True, key=lambda x: x[2])
  if len(fines) > 0 and fines[0][1] == y_test[img][0]:
    count += 1
  for i in range(topk):
    if fines[i][1] ==  y_test[img][0]:
      topkcount += 1
      break

  if img%20 == 0:
    print('\r', img/total, count/(img+1), topkcount/(img+1), '    ', end='')

print(count/total)
print(topkcount/total)